In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


---

The exam has 20 questions.
* Q1 is a conceptual question with 3 parts, each worth 3 points.
* Q2-Q19 is about a dataset of drug overdoses.
    * Q2 is worth 3 points
    * Q3-Q15 are worth 4 points each
    * Q16-Q18 are worth 6 points each
    * Q19 is worth 8 points
* Q20 is a classification question on a dataset of chess games.
    * Part (a) is worth 7 points
    * Part (b) is worth 3 points

*Best of luck!*

---

![Figures](Figures.png)

Consider the two datasets above, where the two colors correspond to two different classes. For the following questions, provide a short reason for your answers. You may use the points named in the figures (from A to P) for your explanations. If there are multiple possible choices for your answer, you can provide any one.

### [Q1(a) 3 points]
Which dataset can I classify better using a depth-1 decision tree (which only allows one top-level split)?

*Fill in*

### [Q1(b) 3 points]
Which dataset can I classify better using a depth-2 decision tree?

*Fill in*

### [Q1(c) 3 points]
Can any linear separator achieve better than 50% accuracy on Dataset 1? Why or why not?

*Fill in*

---

We will consider a dataset (*overdoses.csv*) where each row represents one drug-related death in Connecticut. We will use the following fields:
* Date, Age, Sex, and Race are self-explanatory
* ResidenceCity and ResidenceCounty are the city and county where the deceased person lived.
* DeathCity and DeathCounty are the city and county where the death was reported.
* COD is a text string for the Cause Of Death
* Location is the place where the person died. For example, it could be a Hospital.
* LocationifOther is an extra string giving more details about the Location.
* There are many columns for individual drugs. For example, under the column named "Heroin", we have a "Y" if the person had Heroin in their blood. If there was no Heroin in their blood, the value is missing.
    * Sometimes the value may be something other than a "Y". When this happens, we will just think of it as missing.
    * The "Other" column lists drug names instead of a "Y". We will ignore the "Other" column.
* DeathCityGeo and ResidenceCityGeo give latitude and longitude coordinates for the death and residence cities

### [Q2 3 points] Read in overdoses.csv

Print the SECOND row.

In [26]:
overdose = pd.read_csv("overdoses.csv", index_col=0)
overdose.iloc[1]

Date                                        3/21/2013 0:00
DateType                                       DateofDeath
Age                                                   48.0
Sex                                                   Male
Race                                                 Black
ResidenceCity                                      NORWALK
ResidenceCounty                                        NaN
ResidenceState                                         NaN
DeathCity                                          NORWALK
DeathCounty                                      FAIRFIELD
Location                                          Hospital
LocationifOther                                        NaN
DescriptionofInjury                                    NaN
InjuryPlace                                            NaN
InjuryCity                                             NaN
InjuryCounty                                           NaN
InjuryState                                            N


In some of the following questions, we will need to access the columns corresponding to various drugs.
To make it easier, I have created a list called **drug_names** that you should use in your code. 

* You will need this in Q14, Q16, Q18, and Q19.

**Make sure you run this code cell before proceeding**

In [2]:
drug_names = ['Heroin', 'Cocaine', 'Fentanyl', 'FentanylAnalogue', 'Oxycodone', 'Oxymorphone',
              'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad',
              'Morphine_NotHeroin', 'Hydromorphone', 'OpiateNOS', 'AnyOpioid']

### [Q3 4 points] What is the ratio of Males to Females in this dataset?

### [Q4 4 points] How often is the DeathCity the same as the ResidenceCity?

### [Q5 4 points] How many rows are missing either the ResidenceCity or the DeathCity or both?

### [Q6 4 points] Cities per county

Print out the number of unique DeathCity in each DeathCounty.
* Sometimes either DeathCity or DeathCounty is missing. You should not count these missing rows.
* You will find one obviously-wrong value for DeathCounty. Don't worry about it.

### [Q7 4 points] Drugs for age groups

Split the deaths into 5 equal-sized groups based on Age.
* Each group should have (roughly) the same number of people.

Find the most common COD for each age group.

### [Q8 4 points] Among deaths where the Location of death was a person's Residence, find the number of deaths in each ResidenceCity.

Plot the number of such deaths in the top-10 Residence Cities.

### [Q9 4 points] For each Race, find the most common Sex among those who died.

### [Q10 4 points] Among the deaths where the COD is "Cocaine Intoxication", what fraction has a 'Y' under the Cocaine column?

### [Q11 4 points] Among deaths where Cocaine is detected but the COD does not mention 'Cocaine' or 'COCAINE', what is the most common COD?

### [Q12 4 points] Plot number of deaths by Race.

Remove the "Unknown" Race from the plot.

### [Q13 4 points] LocationifOther data check

The LocationifOther column is supposed to be used only if the Location column has the value "Other". Let us check this.

Count the number of rows where LocationifOther is not missing, but Location is not Other.

### [Q14 4 points] What drugs never co-occur with OpiateNOS?

Two drugs co-occur if there is at least one death where both those drugs had a "Y" under their columns.
You should consider all the drugs whose names are listed in **drug_names**.

### [Q15 4 points] Create a column called DeathCityLatLong

Extract the latitude and longitude from the DeathCityGeo column. For example,
* If DeathCityGeo is "Norwalk, CT\n(41.11805, -73.412906)", then the DeathCityLatLong column should have the value "41.11805, -73.412906"

Print out the DeathCityGeo and DeathCityLatLong columns for the first 5 rows.

### [Q16 6 points] Data check

* Find all deaths where the COD has two words, and the last word is Intoxication. Call these "one-COD deaths".
    * For example, if COD is "Cocaine Intoxication", then it is a one-COD death.
    * But if COD is "Cocaine and Heroin Intoxication", it is not a one-COD death.
* For every one-COD death, figure out how many drug_names were found in the person's blood
    * So if there was a "Y" under the columns "Cocaine" and "AnyOpioid", we count that as 2 drugs.
* Find out how many "one-COD deaths" actually had two or more drug_names in the person's blood.

**Note**: You may find that "Heroin" and "AnyOpioid" are both "Y", but the COD only says "Heroin Intoxication". We will count that as two drugs in the person's blood, even though Heroin is actually an Opioid and this is not really a data error.

### [ Q17 6 points] Plot the number of deaths every month.

The x-axis should be the month, and the y-axis the number of deaths (i.e., rows) for that month.

### [Q18 6 points] Drug Cocktails
Look at deaths where COD mentions the word 'COMBINED'.
Among these deaths, count how many drugs were involved by checking how many of the drug name columns had a 'Y'
Show a bar plot where the x-axis is the number of drugs used, and the y-axis is the number of deaths caused by this drug cocktail.

### [Q19 8 points] Differences between races

* For each DeathCity and Race, figure out how many people had a particular drug in their blood when they died. For example,
    * among Whites in City A, there may be 50 deaths with Cocaine and 12 with Heroin.
    * among Asians in City A, there may be 30 Heroin and 20 Cocaine deaths.
    * among Hispanics in City A, there may be 19 Cocaine and 18 Heroin deaths.
* Now, figure out the top drug which caused the most deaths for each DeathCity and Race. Only consider drugs that killed at least 20 people of that Race in that DeathCity; if there is no such drug, just act as if no one of that Race died in that DeathCity.
    * among Whites in City A, Cocaine is the top drug.
    * among Asians in City A, it is Heroin.
    * among Hispanics in City A, no drug led to at least 20 deaths, so there is no top drug.
* Now, print out the DeathCities for which the top drug is different for at least two Races.
    * Since the top drugs in City A are Cocaine (Whites) and Heroin (Asians), print out City A.
    * But if a City B had Cocaine (Whites) and Cocaine (Asians), don't print out City B.
    * Or if a City C had Heroin (Asians) but no other race had enough deaths to have a top drug, don't print out City C.

---

We will try to predict chess games, using the file *games.csv*.

### [Q20(a) 7 points] Train and test accuracies

* Load in the games.csv file. 
* Split it into training and testing sets, where 80% of the data is used for training. 
* Use a logistic regression classifier to predict whether white will win or not.
    * Check if the "winner" column is "white" or not.
    * You should only use "white_rating", "black_rating" and "opening_name" as features. Don't use any other columns as features.
* Print the training and test accuracies.

### [Q20(b) 3 points] Probability of win

Print the probability of white winning a game where the game has the following features:
* white_rating is 2000
* black_rating is 1500
* All other features are exactly the same as in the first row of the games.csv file.